# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import os
os.chdir('..')

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf

In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies
# For some strange reason PIMC is crashing the second time it is called from Jupyter
models.pimc_use_declaring = False
models.pimc_use_defending = False

Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\models/muppet/512/bidding_V2-1100000
INFO:tensorflow:Restoring parameters from ..\models/contract/contract-193200
INFO:tensorflow:Restoring parameters from ..\models/muppet/binfo_V2-1128000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/single_dummy/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\models/playing/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing

In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'S'
vuln = [True, True]  # fist element is NS, second element is EW

hands = [
    'AJ87632.J96.753.',
    'K9.Q8542.T6.AJ74',
    'QT4.A.KJ94.KQ986',
    '5.KT73.AQ82.T532'
]

auction = ['1N', 'PASS', '4H', 'PASS', '4S', 'PASS', 'PASS', 'PASS']

play = ['C2', 'D3', 'CA', 'C6', 'D6', 'DJ', 'DQ', 'D5', 'DA', 'D7', 'DT', 'D4', 'D8', 'H6', 'H2', 'D9', 'SQ', 'S5', 'S2', 'SK', 'H4', 'HA', 'H7', 'H9', 'S4', 'C3', 'SA', 'S9', 'S3', 'C4', 'ST', 'H3', 'CK', 'C5', 'HJ', 'C7', 'C8', 'CT', 'S6', 'CJ', 'S7', 'H8', 'C9', 'D2', 'S8', 'H5', 'CQ', 'HT', 'SJ', 'HQ', 'DK', 'HK']

In [4]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

In [5]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

await card_by_card.analyze()

analyzing the bidding
1N Suggested bid from NN: CandidateBid(bid=1C  , insta_score=0.9826, expected_score=---, expected_tricks=---, adjust=---, alert=  )
1N NN-values:CandidateBid(bid=1N  , insta_score=0.0170, expected_score=---, expected_tricks=---, adjust=---, alert=  )
PASS OK NN-value: 1.000
4H OK NN-value: 0.996


Loaded lib dds.dll


PASS OK NN-value: 0.926
4S OK NN-value: 0.999
PASS OK NN-value: 1.000
PASS OK NN-value: 1.000
PASS OK NN-value: 1.000
analyzing opening lead
C2
C2 OK
analyzing play
play_card
8
D3 ? losing: 0.26
play_card
49
CA ?? losing: 0.73
play_card
47
C6 OK
play_card
21
D6 OK
play_card
23
DJ OK
play_card
13
DQ OK
play_card
24
D5 OK
play_card
40
DA OK
play_card
50
D7 OK
play_card
12
DT OK
play_card
46
D4 OK
play_card
18
D8 OK
play_card
25
H6 OK
play_card
4
H2 OK
play_card
20
D9 OK
play_card
45
SQ ? losing: 0.18
play_card
48
S5 OK
play_card
11
S2 OK
play_card
42
SK OK
play_card
16
H4 OK
play_card
15
HA OK
play_card
10
H7 OK
play_card
17
H9 OK
play_card
44
S4 OK
play_card
43
C3 OK
play_card
7
SA ?? losing: 0.91
play_card
39
S9 OK
play_card
37
S3 OK
play_card
30
C4 OK
play_card
29
ST OK
play_card
28
H3 OK
play_card
9
CK OK
play_card
0
C5 OK
play_card
5
HJ OK
play_card
2
C7 OK
play_card
35
C8 OK
play_card
34
CT OK
play_card
27
S6 OK
play_card
26
CJ OK
play_card
32
S7 OK
play_card
33
H8 OK
play_card
22


In [6]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '1N',
 'who': 'Analysis',
 'quality': 'Good',
 'candidates': [{'call': '1C', 'insta_score': 0.983},
  {'call': '1N', 'insta_score': 0.017}],
 'samples': ['A98x.Q9xxx.Q8.Ax Jxxx.KJTx.xxx.xx QTx.A.KJ9x.KQ98x Kx.8xx.ATxx.JTxx 0.75000',
  '98xx.Q8xx.Tx.AJx xx.KT9xx.AQxx.xx QTx.A.KJ9x.KQ98x AKJx.Jxx.8xx.Txx 0.75000',
  'K8xx.QTxxx..ATxx 9xx.Kxx.Axxx.Jxx QTx.A.KJ9x.KQ98x AJx.J98x.QT8xx.x 0.75000',
  '9x.KQJ8xx.8x.Jxx K8x.Txx.Txxx.Axx QTx.A.KJ9x.KQ98x AJxxx.9xx.AQx.Tx 0.75000',
  'AJxx.KQJ8x.Tx.Jx K8xx.9.Q8xxx.Txx QTx.A.KJ9x.KQ98x 9x.Txxxxx.Ax.Axx 0.75000',
  'AKJx.KJT8x.xx.xx 9x.Q9xxx.T8xx.AJ QTx.A.KJ9x.KQ98x 8xxx.xx.AQx.Txxx 0.75000',
  'J8x.K8xxx.x.Txxx AK9x.9xx.Txx.AJx QTx.A.KJ9x.KQ98x xxx.QJTx.AQ8xx.x 0.75000',
  '9xxx.K9xx.Qx.Jxx 8x.QJxx.T8xx.ATx QTx.A.KJ9x.KQ98x AKJx.T8xx.Axx.xx 0.75000',
  'KJ8xx.T8.xx.JTxx Axx.9xxx.AT8x.xx QTx.A.KJ9x.KQ98x 9x.KQJxxx.Qxx.Ax 0.75000',
  '8x.K8xx.A8xx.xxx AJ9xx.QTxxx.Qx.A QTx.A.KJ9x.KQ98x Kxx.J9x.Txx.JTxx 0.75000',
  'Axx.xxx.AQ8.Jxxx Jxx.QT98x.

the engine very confidently opens `1C` and doesn't even consider `1N`

In [7]:
# what about the opening lead? let's see...

card_by_card.cards['C2'].to_dict()

{'card': 'C2',
 'who': '',
 'quality': 'Good',
 'hcp': [6.3, 2.1, 2.3, 2.2, 2.8, 3.2, 3.5, 3.5, 2.9, 3.4, 3.3, 3.3],
 'shape': [8.5, 7.3, 15.3],
 'candidates': [{'card': 'Sx',
   'insta_score': 0.487,
   'expected_tricks_sd': 10.26,
   'p_make_contract': 0.23,
   'expected_score_sd': -469},
  {'card': 'Cx',
   'insta_score': 0.3,
   'expected_tricks_sd': 10.29,
   'p_make_contract': 0.22,
   'expected_score_sd': -474},
  {'card': 'Hx',
   'insta_score': 0.144,
   'expected_tricks_sd': 10.5,
   'p_make_contract': 0.2,
   'expected_score_sd': -490},
  {'card': 'DA',
   'insta_score': 0.052,
   'expected_tricks_sd': 10.45,
   'p_make_contract': 0.13,
   'expected_score_sd': -542}],
 'samples': ['x.KTxx.AQ8x.Txxx AQJ98x.Q9xx.Tx.x xx.J8x.KJxx.J9xx KTxx.Ax.9xx.AKQ8 0.74819',
  'x.KTxx.AQ8x.Txxx AKTxxx.xx.J9x.xx J9x.Q9x.KTx.J98x Q8x.AJ8x.xxx.AKQ 0.74992',
  'x.KTxx.AQ8x.Txxx KJ9xxx.AQx.Jxx.x QT8x.xxx.T9x.QJ9 Ax.J98.Kxx.AK8xx 0.71315',
  'x.KTxx.AQ8x.Txxx KQJ9xx.J9x.KJ.Jx xx.Q8xx.9xxx.Q9x AT8x

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [8]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'S6',
 'who': 'NN-Make',
 'quality': 'Good',
 'hcp': [9.1, 8.8],
 'shape': [2.1, 3.2, 3.3, 4.3, 2.0, 3.2, 3.4, 4.3],
 'candidates': [{'card': 'S6',
   'insta_score': 0.61,
   'expected_tricks_dd': 11.26,
   'p_make_contract': 0.98,
   'expected_score_dd': 649},
  {'card': 'S2',
   'insta_score': 0.61,
   'expected_tricks_dd': 11.26,
   'p_make_contract': 0.98,
   'expected_score_dd': 649},
  {'card': 'S3',
   'insta_score': 0.61,
   'expected_tricks_dd': 11.26,
   'p_make_contract': 0.98,
   'expected_score_dd': 649},
  {'card': 'S7',
   'insta_score': 0.61,
   'expected_tricks_dd': 11.26,
   'p_make_contract': 0.98,
   'expected_score_dd': 649},
  {'card': 'S8',
   'insta_score': 0.03,
   'expected_tricks_dd': 11.26,
   'p_make_contract': 0.98,
   'expected_score_dd': 649},
  {'card': 'SJ',
   'insta_score': 0.01,
   'expected_tricks_dd': 11.15,
   'p_make_contract': 0.98,
   'expected_score_dd': 641},
  {'card': 'D3',
   'insta_score': 0.18,
   'expected_tricks_dd': 10.99,
 

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)

## Analyzing a board played on BBO

In [9]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [10]:
display_lin(lin)

In [11]:
board = parse_lin(lin)
print(board)

Board(dealer='S', vuln=[False, True], hands=['9872.A3.KJ973.53', 'AJ6Q.8562.5.4J2Q', 'KT4.J.Q86.AKT976', '53.KQT974.AT42.8'], auction=['1C', '2H', 'X', '4H', '4S', 'PASS', 'PASS', 'PASS'], play=['DA', 'D3', 'D5', 'D6', 'C8', 'C3', 'CJ', 'CA', 'S4', 'S5', 'S8', 'SJ', 'H5', 'HJ', 'HQ', 'HA', 'S2', 'SA', 'ST', 'S3', 'H2', 'SK', 'H4', 'H3', 'D8', 'D2', 'DJ', 'S6', 'SQ', 'C6', 'H7', 'S7', 'H8', 'C7', 'HK', 'S9', 'C5', 'C2', 'CT', 'HT', 'CK', 'H9', 'D7', 'C4', 'DQ', 'D4', 'DK', 'H6', 'D9', 'CQ', 'C9', 'DT'])


In [12]:
card_by_card = CardByCard(*board, models, sampler, False)


In [13]:
await card_by_card.analyze()

analyzing the bidding
1C OK NN-value: 1.000
2H Suggested bid from NN: CandidateBid(bid=3H  , insta_score=0.6901, expected_score=    5, expected_tricks= 9.13, adjust=  35, alert=  )
2H NN-values:CandidateBid(bid=2H  , insta_score=0.3028, expected_score=  -16, expected_tricks= 8.80, adjust=  15, alert=  )
X OK NN-value: 1.000
4H Suggested bid from NN: CandidateBid(bid=3H  , insta_score=0.5616, expected_score=   44, expected_tricks= 9.04, adjust=  28, alert=  )
4H NN-values:CandidateBid(bid=4H  , insta_score=0.3473, expected_score=  -18, expected_tricks= 8.77, adjust=  17, alert=  )
4S OK NN-value: 0.048
PASS OK NN-value: 0.998
PASS OK NN-value: 1.000
PASS Suggested bid from NN: CandidateBid(bid=X   , insta_score=0.3572, expected_score=   58, expected_tricks= 8.83, adjust=  18, alert=  )
PASS NN-values:CandidateBid(bid=PASS, insta_score=0.6419, expected_score=  -12, expected_tricks= 8.83, adjust=  32, alert=  )
analyzing opening lead
DA


TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'

the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [ ]:
card_by_card.cards['S4'].to_dict()

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [ ]:
card_by_card.cards['DA'].to_dict()